In [1]:
import os
import requests
from bs4 import BeautifulSoup
import zipfile
import xport, csv
import pandas as pd
from pandas.api.types import infer_dtype
from datetime import datetime
import re

Define the urls to extract from

In [2]:
url1976_2000 = 'https://www.chicagofed.org/banking/financial-institution-reports/commercial-bank-data-complete-1976-2000'
url2001_2010 = 'https://www.chicagofed.org/banking/financial-institution-reports/commercial-bank-data-complete-2001-2010'
url2011_present = 'https://www.chicagofed.org/banking/financial-institution-reports/commercial-bank-structure-data'
root = 'https://www.chicagofed.org/'

let user input start and end year and quarter

In [3]:
start = '199803'
end = '201803'
start =  datetime.strptime(start, '%Y%m')
end =  datetime.strptime(end, '%Y%m')
quarters = (pd.date_range(pd.to_datetime(start), 
                   pd.to_datetime(end) + pd.offsets.QuarterBegin(1), freq='Q')
      .strftime('%y%m')
      .tolist())

In [4]:
if start <= datetime.strptime('200012', '%Y%m'):
    if end <= datetime.strptime('200012', '%Y%m'):
        url_list = [url1976_2000]
    elif end <= datetime.strptime('201012', '%Y%m'):
        url_list = [url1976_2000,url2001_2010]
    else:
        url_list = [url1976_2000,url2001_2010,url2011_present]

elif start <= datetime.strptime('201012', '%Y%m'):
    if end <= datetime.strptime('201012', '%Y%m'):
        url_list = [url2001_2010]
    else:
        url_list = [url2001_2010,url2011_present]

else:
    url_list = [url2011_present]

Extract and clean the relevant links

In [5]:
def extract_links(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    all_hrefs = soup.find_all('a')
    all_links = [link.get('href') for link in all_hrefs]
    zip_files = [dl for dl in all_links if '.zip' in dl or '.ZIP' in dl]
    zip_files = [dl.lower() for dl in zip_files]
    zip_files = [dl[:dl.index('.zip')+len('.zip')] for dl in zip_files]
    return zip_files

download zip files

In [6]:
def download_zip(zip_files):
    #if download folder does not exist, create one
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    for zip_file in zip_files:
        if re.findall("[0-9]{4}",test) in quarters:
            full_url = root + zip_file
            r = requests.get(full_url)
            zip_filename = os.path.basename(zip_file)
            dl_path = os.path.join(download_folder, zip_filename)
            with open(dl_path, 'wb') as z_file:
                z_file.write(r.content)

unzip

In [7]:
def unzip():
    os.chdir(download_folder)
    for item in os.listdir(download_folder):
        if item.endswith('.zip'):
            file_name = os.path.abspath(item) # get full path of files
            zip_ref = zipfile.ZipFile(file_name) # create zipfile object
            zip_ref.extractall(download_folder) # extract file to dir
            zip_ref.close() # close file
            os.remove(file_name) # delete zipped file

In [ ]:
download_folder = 'C:/Users/kwang648/Downloads/banking/'
for url in url_list:
    download_zip(extract_links(url))
    unzip()

Iterate through three urls

In [ ]:
download_folder = 'C:/Users/kwang648/Downloads/banking/'
for url in url_list:
    download_zip(extract_links(url))
    unzip()

Convert text to csv, and merge

In [ ]:
os.chdir('C:/Users/kwang648/Downloads/banking data/Commercial Bank Data - 2011-2023')
for item in os.listdir('C:/Users/kwang648/Downloads/banking data/Commercial Bank Data - 2011-2023'):
    new = pd.read_csv(item, delimiter = "\t")
    break

Convert xpt to csv and merge

In [ ]:
os.chdir('C:/Users/kwang648/Downloads/banking data/Commercial Bank Data - 1976-2010')
for item in os.listdir('C:/Users/kwang648/Downloads/banking data/Commercial Bank Data - 1976-2010'):
    if item.endswith('.xpt'):
        os.system('python -m xport '+item+' > '+item[:item.index('.xpt')]+'.csv')
    elif item.endswith('.XPT'):
        os.system('python -m xport '+item+' > '+item[:item.index('.XPT')]+'.csv')

handle type conversion

In [ ]:
def type_conversion():
    for column in new.columns:
        if infer_dtype(new[column])=='mixed-integer':
            new[column] = pd.to_numeric(new[column], errors='coerce')

In [ ]:
os.chdir('C:/Users/kwang648/Downloads/banking data\call_csv - 2011-2023')
call_report = pd.DataFrame()
progress = 0
for item in os.listdir('C:/Users/kwang648/Downloads/banking data\call_csv - 2011-2023'):
    progress+=1
    print(progress)
    new = pd.read_csv(item,encoding = 'latin-1')
    type_conversion()
    call_report = pd.concat([call_report,new])

1
2
3
4
5
6
7


C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (74,228) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


8


C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (75) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


9


C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (72) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


10
11


C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (72,220) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


12
13


In [ ]:
call_report

,Reporting_Period_End_Date,FDIC_Certificate_Number,OCC_Charter_Number,OTS_Docket_Number,Primary_ABA_Routing_Number,Financial_Institution_Name,Financial_Institution_Address,Financial_Institution_City,Financial_Institution_State,Financial_Institution_Zip_Code,...,RIADH034,RIADH035,RIADHK03,RIADHK04,RIADHT69,RIADHT70,RIADHT73,RIADHT74,RIADJ321,RIADJJ33
0,2011-12-31,10057,0,16553,61107146,BANK OF HANCOCK COUNTY,12855 BROAD STREET,SPARTA,GA,31087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-12-31,3850,0,0,81220537,FIRST COMMUNITY BANK XENIA-FLORA,260 FRONT STREET,XENIA,IL,62899,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-12-31,28868,0,2523,311972526,"MINEOLA COMMUNITY BANK, SSB",215 W BROAD,MINEOLA,TX,75773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-12-31,14083,0,0,101107475,BISON STATE BANK,223 MAIN STREET,BISON,KS,67520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-12-31,10202,0,0,91208332,LOWRY STATE BANK,400 FLORENCE AVE.,LOWRY,MN,56349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4719,2023-03-31,59312,0,0,21315169,GS&L MUNICIPAL BANK,42 CHURCH STREET,GOUVERNEUR,NY,13642,...,NaN,NaN,1.0,1.0,262.0,0.0,0.0,0.0,NaN,0.0
4720,2023-03-31,59289,0,0,11402121,WALDEN MUTUAL BANK,66 N MAIN STREET,CONCORD,NH,3301,...,NaN,NaN,3.0,11.0,-689.0,0.0,0.0,0.0,NaN,76.0
4721,2023-03-31,59296,0,0,122245387,BANK IRVINE,"6201 OAK CANYON, SUITE 150",IRVINE,CA,92618,...,NaN,NaN,2.0,58.0,-797.0,0.0,0.0,0.0,NaN,212.0
4722,2023-03-31,57288,25281,0,122106497,"WESTERN ALLIANCE TRUST COMPANY, NATIONAL ASSOC...",1 EAST WASHINGTON STREET SUITE 1400,PHOENIX,AZ,85004,...,NaN,NaN,0.0,0.0,-3323.0,0.0,0.0,0.0,NaN,0.0
